In [40]:
%load_ext autoreload
%autoreload 2

In [41]:
import torch
import torch.nn as nn
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f'CUDA {i}: {torch.cuda.get_device_name(i)}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CUDA 0: NVIDIA GeForce GTX 1650 Ti


In [55]:
class MLP(nn.Module):
    def __init__(self, width=200, N_layers=10, loss_bc_lambda=1.0, activation=nn.Softplus(beta=1)):
        super().__init__()
        _layers = [nn.Linear(3, width), activation]
        for i in range(N_layers):
            _layers.append(nn.Linear(width, width))
            _layers.append(activation)
        _layers.append(nn.Linear(width, 1))
        self.model = nn.Sequential(*_layers)
        self.loss_function = nn.MSELoss(reduction='mean')
        self.loss_bc_lambda = torch.tensor([loss_bc_lambda], requires_grad=False)

    def forward(self, x):
        return self.model(x)

    def loss_BC(self, x, sdf):
        loss = self.loss_function(self.forward(x), sdf)
        return loss
    
    def loss_PDE(self, x):
        _x = x.clone()
        _x.requires_grad = True
        F = self.forward(_x)
        F_x = torch.autograd.grad(
            F, _x,
            grad_outputs=torch.ones_like(F),
            retain_graph=True,
            create_graph=True
        )[0]
        loss = self.loss_function(torch.abs(F_x), torch.ones_like(F_x))
        return loss
    
    def loss(self, x, sdf):
        return self.loss_PDE(x) + self.loss_bc_lambda * self.loss_BC(x, sdf)



In [56]:
net = MLP()
net.loss(torch.rand(1000, 3), torch.rand(1000,1))

tensor([2.3634], grad_fn=<AddBackward0>)